In [11]:
algo_3_output_folder = 'algorithm_output'
retrieved_filename = 'retrieved_result.csv'
# extraction_keys_filename = 'environstd_inchi.csv'
# extraction_keys_file_encoding = 'ANSI'

foldername = 'GCMS_output' # this is the forlder where the cleaned data is stored
cleaned_folder_name = 'RIgrouped_data'

read_folder = os.path.join(os.getcwd(), foldername)
cleaned_folder = os.path.join(read_folder, cleaned_folder_name) # folder that we will be reading data from
# df = pd.read_csv(os.path.join(os.getcwd(), extraction_keys_filename), encoding=extraction_keys_file_encoding)
# df.head()

In [12]:
import os
import glob
from tqdm import tqdm
import pandas as pd
import ast

In [13]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [14]:
# inchikey_list = df['InChIKey:'].str.strip().array
# name_list = df['Name:'].str.strip().array
columns_to_be_included = [
    'Name:', 
    'Formula:',
    'MW:',
    'NIST#:',
    'InChIKey:', 
    'MS2:',
    'Score:',
    'EstRI', #try to extract the 2 RI values in the text
    'PredRI'
]

In [15]:
#algorithm part from algorithm3
##clean up the mw to valide python datatype
#found some faulty data in the result datafile and modify the codes  August 4
#work on 2021 August16 for modification for the RI grouped data
def clean_MW(df):
# def clean_precursormz(df):
    inchikeys = df['InChIKey:'].unique().tolist()
    for inchikey in inchikeys:
        curr_precursormz = df.loc[df['InChIKey:'] == inchikey, 'MW:'].iloc[0]
        try:
            curr_precursormz = ast.literal_eval(curr_precursormz)
            df.loc[df['InChIKey:'] == inchikey, ['MW:']] = curr_precursormz[0]
        except TypeError:
            curr_precursormz = float(curr_precursormz)
            df.loc[df['InChIKey:'] == inchikey, ['MW:']] = curr_precursormz
        except ValueError:
            curr_precursormz = float(curr_precursormz)
            df.loc[df['InChIKey:'] == inchikey, ['MW:']] = curr_precursormz
    return df

def retrieve_ms2_dict(df): 
    ms2_dict = pd.Series(df['Score:'].array, index=df['MS2:']).to_dict()
    return ms2_dict

def filter_score(df, threshold):
    return df[df['Score:'] >= threshold]

def filter_ms2(df, msthreshold):
    return df[df['MS2:'] >= msthreshold]

def append_to_list(df):
    output_dict = {}
    for i, item in df['Name:'].items():
        if 'Name:' in output_dict.keys():
            if item not in output_dict['Name:']:
                if type(output_dict['Name:']) == list:
                    if type(item) == list:
                        output_dict['Name:'] += item
                    else:
                        output_dict['Name:'].append(item)
                else:
                    if type(item) == list:
                        output_dict['Name:'] = item.append(output_dict['Name:'])
                    else:
                        output_dict['Name:'] = [output_dict['Name:'], item]
        else:
            output_dict['Name:'] = item
    return pd.Series(output_dict)
#saving compound name in the 'Name:' column, some list of names inside the column

# markdown: 

In [16]:
import time
#set search range for the similar product ms
search_range = 1
threshold =100 #threshold for ms2 score
msthreshold = 10 #threshold for ms2 

output_folder = os.path.join(os.getcwd(), algo_3_output_folder, 'non_combined_output')

if not (os.path.exists(output_folder)):
    os.makedirs(output_folder)

## get list of csv file names
cwd = os.getcwd()
os.chdir(cleaned_folder)
listdir = glob.glob('*.csv')
os.chdir(cwd)

In [ ]:
#from below, precursormz === mw  
# start = time.time()
for filename in tqdm(listdir):
    precursormz = int(filename.split('_')[0])  
    cleaned_file = os.path.join(cleaned_folder, filename)
    cleaned_df = pd.read_csv(cleaned_file)
    #sort the datafile to pop the most abundance ms2 to the top 
    cleaned_df.sort_values(by=['InChIKey:', 'Score:'], inplace=True)     
    #clean up the precursor ion or the mw, make it into python datatype
    cleaned_df = clean_MW(cleaned_df)
    cleaned_df['Score:'] = pd.to_numeric(cleaned_df['Score:'])
    cleaned_df['MS2:'] = pd.to_numeric(cleaned_df['MS2:'])
    cleaned_df = filter_score(cleaned_df, threshold)  ##limits of potential product ions for calculation in a short time
    cleaned_df = filter_ms2(cleaned_df, msthreshold)  # cleaned_df = filter_ms2(cleaned_df, msthreshold)  #
    
#calculate the precision and recall inside one file
    output_file = os.path.join(output_folder, str(precursormz) + '_algo3.csv')
    inchikeys = cleaned_df['InChIKey:'].unique().tolist()
#if no interfering items mean the compound is only one detected which is faulty data, break out the loop
    if len(inchikeys) <=1:
        continue
    
    TP, FN = 1,0
    header_flag = True
    
    end1 = time.time()
#     print("First elapsed:", end1-start)

    for inchikey in inchikeys:
#put all ms2 lists of one compound all in the target_item_df by inchikey separation 
        target_item_df = cleaned_df[cleaned_df['InChIKey:'] == inchikey].copy()  
        interfering_items_df = cleaned_df[cleaned_df['InChIKey:'] != inchikey]
            
        interfering_items_s = interfering_items_df.groupby(['InChIKey:']).apply(retrieve_ms2_dict)
        target_ms2_array = target_item_df['MS2:'].array
        output_dict = {}
        end2 = time.time()
#         print("Second elapsed:", end2-end1)
        
#start the tn fp calculation#
# ====================================================================================================================== #
        for ms2 in target_ms2_array:
                FP, TN = 0, 0
                for i, ms2_dict in interfering_items_s.iteritems():
                    TN_flag = True
                    
                    if (int(ms2) - int(max(ms2_dict.keys()))) > 1:
                        TN += 1
                        continue

#make keys into array and using broadcasting by minus to count the number of [-1,0,1]
#make ms2 array
#                     a = np.array([ms2])
#                     b = np.array([*ms2_dict])
#                     result = np.subtract(b,a)
#get rid of the for loop with if conditional sentences
#count the number of [-1,0,1]
#                     count = 0
#                     for i in range(len(result)):
#                         if result[i] == 0:
#                             count +=1
#                         elif result [i] == 1:
#                             count +=1
#                         elif result [i] == -1:
#                             count +=1
#                     unique, counts = np.unique(result, return_counts = True)
#                     resultdict = dict(zip(unique,counts))
#                     count=0
#                     if 1 in resultdict.keys():
#                         count+=1
#                     elif 0 in resultdict.keys():
#                         count +=1
#                     elif -1 in resultdict.keys():
#                         count+=1
#                     else:
#                         continue
                        
#                     if count >=1:
#                         FP+=1
#                     else:
#                         TN+=1
#                     FP = resultdict['-1'] + resultdict['0'] + resultdict['1']
#                     TN = N - FP
                    # N is the number of ms2 list in the interfering target dataframes  
                    
#                     end3 = time.time()
#                     print("Third elapsed:", end3-end2)
                    for key in ms2_dict.keys():
                        diff = round(int(ms2) - int(key), 2)
                        if abs(diff) <= search_range:
                            TN_flag = False
                            FP += 1
                        elif diff < 0:
                            if (TN_flag):
                                TN += 1
                            break
                
#                 end4 = time.time()
#                 print("4th elapsed:", end4-end3)
                output_dict[ms2] = {
                'TP': TP,
                'FN': FN,
                'TN': TN,
                'FP': FP,
                'Accuracy': (TP+TN) / (TP+TN+FP+FN),
                'Sensitivity': TP / (TP+FN),
                'Specificity': TN / (FP+TN)
                }
        output_df = pd.DataFrame.from_dict(output_dict, orient='index')
        output_df = target_item_df.join(output_df, on='MS2:')

        output_df.sort_values(by=['Specificity'], inplace=True)
        output_df.to_csv(output_file, index=False, mode='a', header=header_flag)
        header_flag = False #only include header for the first time

In [ ]:
#the algoirthm for the whole dataset need 67hrs to finish the calculation 2022-3-18 17:42

In [ ]:
# combine all output into same csv file
read_folder = os.path.join(os.getcwd(), algo_3_output_folder, 'non_combined_output')
output_folder = os.path.join(os.getcwd(), algo_3_output_folder)
cwd = os.getcwd()
os.chdir(read_folder)
listdir = glob.glob('*.csv')
os.chdir(cwd)

output_filename = os.path.join(output_folder, 'combined_output_algo_3.csv')

combined_csv = pd.concat([ pd.read_csv(os.path.join(read_folder, f)) for f in tqdm(listdir) ], ignore_index=True, sort=False)
combined_csv.to_csv(output_filename, index=False)

In [ ]:
##remove the repeated inchikey and reoutput


In [ ]:
#retreive compounds from the combined result 

In [ ]:
# algo_3_output_folder = 'algorithm_3_output'
retrieved_filename = 'retrieved_result.csv'
extraction_keys_filename = 'availchem_inchi.csv'
extraction_keys_file_encoding = 'ANSI'
algo_3_result_filename = 'combined_output_algo_3.csv'

df = pd.read_csv(os.path.join(os.getcwd(), extraction_keys_filename), encoding=extraction_keys_file_encoding)
df.columns = ['name','cas','smiles','mw','InChiKey']
inchikey_list = df['InChiKey'].str.strip().array
name_list = df['name'].str.strip().array

In [ ]:
# retrieve result from inchikey list

algo_3_output_folder = os.path.join(os.getcwd(), algo_3_output_folder)
retrieved_output_filename = os.path.join(algo_3_output_folder, retrieved_filename)

df = pd.read_csv(os.path.join(algo_3_output_folder, algo_3_result_filename))
    
output_df = df[df['InChIKey:'].isin(inchikey_list)]
# for name in name_list:
#     try:
#         #append isomers by adding different name
#         output_df = output_df.append(df[df['Name:'].str.contains(name, case=True, regex=False)], ignore_index=True)
#     except KeyError:
#         pass
    
output_df.drop(labels=['TP', 'FP', 'TN', 'FN'], inplace=True, axis=1)
output_df.to_csv(retrieved_output_filename, index=False)